In [3]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load datasets
features_df = pd.read_csv('train.csv')
labels_df = pd.read_csv('train_labels.csv')

# Merge on date_id
data = pd.merge(features_df, labels_df[['date_id', 'target_0']], on='date_id')

# Prepare features and target
X = data.drop(['date_id', 'target_0'], axis=1)
y = data['target_0']

# Skip rows with NaN values
mask = X.notnull().all(axis=1) & y.notnull()
X = X[mask]
y = y[mask]

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)


In [4]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

# Prepare LightGBM dataset
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# Set parameters
params = {
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'verbose': -1
}

# Train LightGBM model
lgbm_model = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=[lgb_train, lgb_eval], early_stopping_rounds=10, verbose_eval=False)

# Predict and evaluate
y_pred_lgbm = lgbm_model.predict(X_test, num_iteration=lgbm_model.best_iteration)
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
print(f"LightGBM Mean Squared Error: {mse_lgbm:.4f}")

# Random Forest Model
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Random Forest Mean Squared Error: {mse_rf:.4f}")

TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'